In [1]:
# !pip install python-docx

In [2]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string
import os
import zipfile
import re
from docx import Document


In [3]:
stop_words = set(stopwords.words('english')) 

In [4]:
zip_filename = 'text_files.zip'
extract_path ='text_files'
file_types=['.txt', '.docx']

In [5]:
with zipfile.ZipFile(zip_filename, "r") as zip_ref:
    zip_ref.extractall()    

In [6]:
paths = []
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if any([file.endswith(ft) for ft in file_types]):
            path = (os.path.join(root, file), file)
            paths.append(path)


In [7]:
def process_file(file_name):
    """"""
    unique_words = []
    fname, file_extension = os.path.splitext(file_name)
    if file_extension== '.txt':
        f = open(file_name, 'r', encoding="utf8")
        lines = f.read().strip()
    else:
        document = Document(file_name)
        lines = ' '.join([p.text for p in document.paragraphs]).strip()
    # remove urls
    url=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'
    lines = re.sub(url, '', lines, flags=re.MULTILINE)
    # remove phone numbers
    phone = '(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})'
    lines = re.sub(phone, '', lines, flags=re.MULTILINE)
    # remove punctuation
    lines = lines.encode("ascii", "ignore")
    lines = lines.decode()
    lines = lines.translate(str.maketrans('','',string.punctuation)).strip()
    # remove stop words
    pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
    lines = pattern.sub('', lines.lower())
    # unique_words = sorted(list(set(lines.split())))
    unique_words = sorted(list(set(word_tokenize(lines))))
    # remove single character which might considered as stop words 
    unique_words = [s for s in unique_words if len(s)>1]
    return unique_words

In [8]:
obj = {}

for path, file in paths:
    print ('Processing: ', file)
    obj[file] = process_file(path)
    
    
print('Saving File')
with open('output_part1.dat', "w") as f:
    for key in obj.keys():
        line = '"'+key +'"'+ ':' + ','.join(obj[key]) + '\n'
        f.write(line)
f.close()
print('Process Finished')

Processing:  52256-0.txt
Processing:  53031-0.txt
Processing:  58108-0.txt
Processing:  blind_text.txt
Processing:  dr_yawn.txt
Processing:  how_rubber_goods_are_made.txt
Processing:  most_boring_ever.txt
Processing:  most_boring_part2.txt
Processing:  pg12814.txt
Processing:  pg14895.txt
Processing:  pg43994.txt
Processing:  random_text.txt
Processing:  smiley_the_bunny.txt
Processing:  week_10_document1.docx
Processing:  week_10_document2.docx
Saving File
Process Finished
